# Optimizing eBay Sales with Predictive Analytics: 
###  Analysing Trends and Forecasting Future Performance!

![image.png]()

# Business Understanding

### Overview

eBay is an eCommerce platform that operates as an online marketplace where individuals and businesses can buy and sell a wide variety of products, including electronics, fashion, collectibles, home goods, and more.

**Stakeholders**
The primary stakeholders in this project include:
- Executive Leadership (eBay C-Suite): They require accurate sales forecasts to make informed decisions about resource allocation and market positioning.
- Inventory Management Team: They need accurate predictions to avoid stockouts or overstocks, which can lead to lost sales or excess inventory costs.
- Marketing Team:  They need to understand sales trends to time campaigns effectively and allocate budget to the most promising opportunities.
- Finance Department: Accurate sales forecasts are critical for their planning processes, ensuring that revenue projections align with actual business performance.
- Investors: They rely on accurate sales predictions to gauge the company's future performance, which influences their investment decisions and perceptions of eBay's market value.

### Problem Statement
In the highly competitive eCommerce landscape, accurately predicting sales is crucial for optimizing inventory management, marketing strategies, and overall business performance. eBay, as a major online marketplace, experiences significant fluctuations in sales due to various factors including seasonal trends, promotional activities, market demand, and external economic conditions.
> **The challenge** fluctuations in sales due to various factors including seasonal trends, promotional activities, market demand, and external economic conditions. This leads to poor decision-making, Inefficient resource allocation, and poor financial planning.

### Objectives 
**Main objective**: To create a model that predicts eBay sales accurately. This will help with better inventory management, marketing, and financial planning by using past sales data and other factors to make informed business decisions.

**Specific Objectives**: 
- Develop a predictive model capable of forecasting eBay sales with a high degree of accuracy on a weekly or monthly basis.
- Demonstrate the practical application of the predictive model in optimizing inventory management by reducing stockouts and overstocks. 
- Identify opportunities for targeted marketing campaigns based on predicted sales trends.

### Success Metric

Evaluate the performance of the predictive model using appropriate metrics such as Mean Absolute Error (MAE), Mean Squared Error (MSE), Root Mean Squared Error (RMSE), and Mean Absolute Percentage Error (MAPE)

### Conclusion

By developing and implementing a robust predictive model, eBay can significantly enhance its ability to forecast sales, optimize inventory management, and make data-driven marketing decisions.

# Data Understanding

**Data Collection**
Source: Yahoo Finance via yfinance API.

**Data Components**

**Historical Stock Prices:** Includes Open, High, Low, Close prices, and Volume of trades.These key features provide a snapshot of a stock's performance on any given trading day:
- Open: It is the first price at which a transaction is made in that stock during the day.
- High: The highest price at which the stock traded during the day. 
- Low: The lowest price at which the stock traded during the day. 
- Close: The final price at which the stock traded when the market closed. 
- Volume: The total number of shares traded during the day. 

**Additional Features:** Adjusted Close prices for accounting for corporate actions like stock splits and dividends.
The Adjusted Close price adjusts the closing price to reflect the impact of corporate actions like dividends, stock splits, and new stock issuance. This feature is essential for long-term analysis because it provides a more accurate picture of a stock’s value over time.
- Dividends: When a company pays out dividends to its shareholders, the stock price typically drops by the dividend amount. The Adjusted Close accounts for this, ensuring that historical price data remains consistent and comparable.
- Stock Splits: In a stock split, a company increases the number of its shares without changing the market value of the company. For instance, in a 2-for-1 split, the stock price is halved, but the number of shares doubles. The Adjusted Close adjusts for this, so the price before and after the split can be compared directly.

In [ ]:
!pip install yfinance

In [2]:
# Import libraries
import yfinance as yf

# Define the ticker symbol for eBay
ticker_symbol = 'EBAY'

# Fetch historical market data for eBay
ebay_data = yf.Ticker(ticker_symbol)

# Get historical data (you can specify the period and interval)
historical_data = ebay_data.history(period='max', interval='1d')

# Save the historical data to a CSV file
historical_data.to_csv('ebay_historical_data.csv')

#print the first few rows to confirm
historical_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1998-09-24 00:00:00-04:00,0.855242,0.867232,0.707373,0.757329,518656090,0.0,0.0
1998-09-25 00:00:00-04:00,0.727356,0.737347,0.657418,0.717364,121255834,0.0,0.0
1998-09-28 00:00:00-04:00,0.759328,0.813279,0.735349,0.771317,89772883,0.0,0.0
1998-09-29 00:00:00-04:00,0.789300,0.803289,0.763324,0.781308,39289536,0.0,0.0
1998-09-30 00:00:00-04:00,0.754331,0.767320,0.691388,0.720361,44113766,0.0,0.0
...,...,...,...,...,...,...,...
2024-08-02 00:00:00-04:00,55.709999,56.849998,55.340000,56.200001,5578600,0.0,0.0
2024-08-05 00:00:00-04:00,54.900002,56.209999,54.549999,54.660000,7416200,0.0,0.0
2024-08-06 00:00:00-04:00,55.000000,56.650002,54.560001,55.919998,4492200,0.0,0.0


# Data Preparation

# Modelling

# Evaluation

# Conclusion and Recommendation